# Personas in Space

In post 020, we tracked persona drift in text-only conversations. But what happens when agents exist in a **2D spatial world**? Does spatial context affect persona stability?

This post combines:
- **2D world simulation** (from post 017): Agents move and interact spatially
- **Persona drift tracking** (from post 020): ConvoKit metrics over time
- **30 iterations**: Enough to see drift patterns
- **2 and 4 speakers**: Compare small vs medium groups

## The Question

Does spatial context make personas more or less stable? Does proximity affect how agents communicate? Do agents develop spatial behaviors that influence their personas?


## Setup and Imports


In [ ]:
import os
import re
from typing import List, Dict
from collections import defaultdict
from dataclasses import dataclass, field
from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm
from random import shuffle, choice, random
import numpy as np
import matplotlib.pyplot as plt

# ConvoKit imports
from convokit import Corpus, Utterance, Speaker
from convokit.text_processing import TextParser
from convokit.politeness_strategies import PolitenessStrategies
from convokit.coordination import Coordination

load_dotenv("../../.env")
client = OpenAI()

GRID_SIZE = 20
TOPIC = "Code, testing, and infra as a source of truth versus comprehensive documentation."


## 2D World Agent Class

We'll use the Agent class from post 017 that supports spatial movement.


In [ ]:
@dataclass
class Agent:
    name: str
    x: int
    y: int
    color: str
    history: list = field(default_factory=list)
    
    def move(self, dx, dy):
        self.x = max(0, min(GRID_SIZE-1, self.x + dx))
        self.y = max(0, min(GRID_SIZE-1, self.y + dy))
    
    def get_position(self):
        return (self.x, self.y)
    
    def distance_to(self, other):
        """Calculate Euclidean distance to another agent."""
        return np.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)


## 2D World Conversation Runner

This combines spatial movement with conversation, tracking both over 30 iterations.


In [ ]:
def run_2d_world_conversation(
    iterations: int,
    participant_count: int,
    evaluation_interval: int = 5,
) -> tuple[List[Dict], List[Dict], List[Dict]]:
    """
    Run conversation in 2D world with spatial movement.
    Returns (conversation_history, metric_history, position_history)
    """
    conversation_history = []
    metric_history = []
    position_history = []
    
    # Initialize agents in 2D space
    agents = []
    for i in range(participant_count):
        # Spread agents across the grid
        x = random.randint(0, GRID_SIZE - 1)
        y = random.randint(0, GRID_SIZE - 1)
        agents.append(Agent(f"speaker_{i+1}", x, y, f"color_{i+1}"))
    
    identity_summaries = {}
    
    # Bootstrap identities
    for agent in agents:
        bootstrap_messages = [
            {
                "role": "system",
                "content": (
                    f"You are {agent.name} in a group conversation among "
                    "experienced software engineers in a 2D virtual world. "
                    "You can move around and interact with others. "
                    "Imagine your own background, priorities, and communication "
                    "style. First, in 2-3 sentences, describe who you are and "
                    "what you care about as an engineer. Then start sharing your "
                    "perspective on the topic below."
                ),
            },
            {"role": "user", "content": f"The topic is: {TOPIC}"},
        ]
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=bootstrap_messages,
            store=False,
        )
        first_message = response.choices[0].message.content
        identity_summaries[agent.name] = first_message
        conversation_history.append(
            {"role": "assistant", "name": agent.name, "content": first_message}
        )
    
    # Record initial positions
    position_history.append({
        agent.name: (agent.x, agent.y) for agent in agents
    })
    
    def get_agent_action(agent: Agent, all_agents: List[Agent], transcript: List[str]):
        """Get action for an agent in the 2D world."""
        others = [a for a in all_agents if a != agent]
        others_loc = "\\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
        
        # Calculate distances
        nearby = [a for a in others if agent.distance_to(a) <= 5]
        nearby_info = "\\n".join([f"- {a.name}: ({a.x}, {a.y}), distance {agent.distance_to(a):.1f}" for a in nearby])
        if not nearby_info:
            nearby_info = "None (you are alone)"
        
        system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}

Nearby avatars (within 5 units):
{nearby_info}

Recent chat messages:
{chr(10).join(transcript[-3:]) if transcript else "No messages yet."}

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - the grid, your position, or any topic you want

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences).
"""
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt}]
        )
        content = response.choices[0].message.content.strip()
        
        # Parse movement
        direction = None
        match = re.search(r'\\[MOVE:\\s*(\\w+)\\]', content)
        if match:
            direction = match.group(1).upper()
            if direction == "UP": agent.move(0, 1)
            elif direction == "DOWN": agent.move(0, -1)
            elif direction == "LEFT": agent.move(-1, 0)
            elif direction == "RIGHT": agent.move(1, 0)
        
        # Extract message
        message = re.sub(r'\\[MOVE:\\s*\\w+\\]', '', content).strip()
        if message:
            transcript.append(f"{agent.name}: {message}")
            conversation_history.append(
                {"role": "assistant", "name": agent.name, "content": message}
            )
    
    # Run iterations
    transcript = []
    for i in tqdm(range(iterations), desc=f"Running {participant_count}-speaker 2D conversation"):
        # Shuffle agent order each round
        shuffled_agents = agents.copy()
        random.shuffle(shuffled_agents)
        
        # Each agent acts
        for agent in shuffled_agents:
            if random.random() < 0.7:  # 70% chance to act
                get_agent_action(agent, agents, transcript)
        
        # Record positions
        position_history.append({
            agent.name: (agent.x, agent.y) for agent in agents
        })
        
        # Evaluate metrics at intervals
        if (i + 1) % evaluation_interval == 0:
            # Evaluate using the function defined below
            corpus = conversation_to_corpus(conversation_history)
            metrics = {
                "dynamic": compute_dynamic_score(corpus),
                "conclusiveness": compute_conclusiveness_score(corpus),
                "speaker_identity": compute_speaker_identity_score(corpus),
                "speaker_fluidity": compute_speaker_fluidity_score(corpus),
                "round": i + 1
            }
            metric_history.append(metrics)
    
    return conversation_history, metric_history, position_history


In [ ]:
def conversation_to_corpus(conversation_history: List[Dict]) -> Corpus:
    """Convert conversation history to a ConvoKit Corpus."""
    utterances = []
    
    for idx, msg in enumerate(conversation_history):
        if msg.get("role") == "assistant" and "name" in msg:
            speaker_id = msg["name"]
            text = msg["content"]
            
            utterance = Utterance(
                id=f"utt_{idx}",
                speaker=Speaker(id=speaker_id),
                text=text
            )
            utterance.meta["timestamp"] = idx
            utterances.append(utterance)
    
    return Corpus(utterances=utterances)

def compute_dynamic_score(corpus: Corpus) -> float:
    """Dynamic: Collaborative (1) vs. Competitive (10)"""
    try:
        parser = TextParser()
        text_corpus = parser.transform(corpus)
        ps = PolitenessStrategies()
        ps_corpus = ps.transform(text_corpus)
        
        politeness_scores = []
        for utt in ps_corpus.iter_utterances():
            ps_score = utt.meta.get("politeness_strategies", {})
            positive_markers = sum([
                ps_score.get("feature_politeness_==HASPOSITIVE==", 0),
                ps_score.get("feature_politeness_==HASNEGATIVE==", 0) * -1,
            ])
            politeness_scores.append(positive_markers)
        
        avg_politeness = np.mean(politeness_scores) if politeness_scores else 0
        avg_politeness_normalized = min(1.0, max(0.0, avg_politeness / 5.0))
        
        speaker_counts = defaultdict(int)
        for utt in corpus.iter_utterances():
            speaker_counts[utt.speaker.id] += 1
        
        if len(speaker_counts) == 0:
            return 5.0
        
        total = sum(speaker_counts.values())
        probs = [count / total for count in speaker_counts.values()]
        entropy = -sum(p * np.log2(p) for p in probs if p > 0)
        max_entropy = np.log2(len(speaker_counts))
        balance_score = entropy / max_entropy if max_entropy > 0 else 0
        
        combined = (avg_politeness_normalized + balance_score) / 2
        score = 10 - (combined * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing dynamic score: {e}")
        return 5.0

def compute_conclusiveness_score(corpus: Corpus) -> float:
    """Conclusiveness: Consensus (1) vs. Divergence (10)"""
    try:
        coord = Coordination()
        coord_corpus = coord.fit_transform(corpus)
        
        agreement_markers = ["agree", "yes", "exactly", "right", "true", "correct", "indeed", "absolutely", "definitely"]
        disagreement_markers = ["disagree", "no", "but", "however", "although", "wrong", "incorrect", "dispute", "differ"]
        
        agreement_count = 0
        disagreement_count = 0
        
        for utt in coord_corpus.iter_utterances():
            text_lower = utt.text.lower()
            for marker in agreement_markers:
                if marker in text_lower:
                    agreement_count += 1
            for marker in disagreement_markers:
                if marker in text_lower:
                    disagreement_count += 1
        
        if agreement_count == 0 and disagreement_count == 0:
            return 5.0
        elif disagreement_count == 0:
            return 1.0
        elif agreement_count == 0:
            return 10.0
        else:
            agreement_ratio = agreement_count / disagreement_count
        
        if agreement_ratio >= 2:
            score = 1 + (1 / (agreement_ratio - 1 + 1)) * 4
        elif agreement_ratio <= 0.5:
            score = 10 - (agreement_ratio * 4)
        else:
            score = 5.0
        
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing conclusiveness score: {e}")
        return 5.0

def compute_speaker_identity_score(corpus: Corpus) -> float:
    """Speaker Identity: Similarity (1) vs. Diversity (10)"""
    try:
        speakers = {}
        
        for utt in corpus.iter_utterances():
            speaker_id = utt.speaker.id
            if speaker_id not in speakers:
                speakers[speaker_id] = {"words": set()}
            
            words = set(re.findall(r'\\b\\w+\\b', utt.text.lower()))
            speakers[speaker_id]["words"].update(words)
        
        if len(speakers) < 2:
            return 5.0
        
        speaker_list = list(speakers.keys())
        overlaps = []
        
        for i in range(len(speaker_list)):
            for j in range(i + 1, len(speaker_list)):
                words_i = speakers[speaker_list[i]]["words"]
                words_j = speakers[speaker_list[j]]["words"]
                
                if len(words_i) == 0 or len(words_j) == 0:
                    continue
                
                overlap = len(words_i & words_j) / len(words_i | words_j)
                overlaps.append(overlap)
        
        if not overlaps:
            return 5.0
        
        avg_overlap = np.mean(overlaps)
        score = 10 - (avg_overlap * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing speaker identity score: {e}")
        return 5.0

def compute_speaker_fluidity_score(corpus: Corpus, window_size: int = 10) -> float:
    """Speaker Fluidity: Malleability (1) vs. Consistency (10)"""
    try:
        speaker_utterances = defaultdict(list)
        
        for utt in corpus.iter_utterances():
            speaker_utterances[utt.speaker.id].append({
                "text": utt.text,
                "timestamp": utt.meta.get("timestamp", 0)
            })
        
        if len(speaker_utterances) == 0:
            return 5.0
        
        consistency_scores = []
        
        for speaker_id, utts in speaker_utterances.items():
            if len(utts) < window_size:
                continue
            
            utts_sorted = sorted(utts, key=lambda x: x["timestamp"])
            mid_point = len(utts_sorted) // 2
            
            first_half_words = set()
            second_half_words = set()
            
            for utt in utts_sorted[:mid_point]:
                words = set(re.findall(r'\\b\\w+\\b', utt["text"].lower()))
                first_half_words.update(words)
            
            for utt in utts_sorted[mid_point:]:
                words = set(re.findall(r'\\b\\w+\\b', utt["text"].lower()))
                second_half_words.update(words)
            
            if len(first_half_words) == 0 or len(second_half_words) == 0:
                continue
            
            overlap = len(first_half_words & second_half_words)
            union = len(first_half_words | second_half_words)
            similarity = overlap / union if union > 0 else 0
            consistency_scores.append(similarity)
        
        if not consistency_scores:
            return 5.0
        
        avg_consistency = np.mean(consistency_scores)
        score = 1 + (avg_consistency * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing speaker fluidity score: {e}")
        return 5.0

def evaluate_conversation(conversation_history: List[Dict]) -> Dict[str, float]:
    """Compute all 4 evaluation metrics using ConvoKit."""
    corpus = conversation_to_corpus(conversation_history)
    
    return {
        "dynamic": compute_dynamic_score(corpus),
        "conclusiveness": compute_conclusiveness_score(corpus),
        "speaker_identity": compute_speaker_identity_score(corpus),
        "speaker_fluidity": compute_speaker_fluidity_score(corpus)
    }


In [ ]:
# Run 2-speaker conversation in 2D world
print("Running 2-speaker conversation in 2D world...")
conv_2d_2speaker, metrics_2d_2speaker, positions_2d_2speaker = run_2d_world_conversation(
    iterations=30,
    participant_count=2,
    evaluation_interval=5
)

print("\\n=== 2-Speaker 2D World Results ===")
for metric_point in metrics_2d_2speaker:
    round_num = metric_point["round"]
    print(f"\\nRound {round_num}:")
    print(f"  Dynamic: {metric_point['dynamic']:.2f}/10")
    print(f"  Conclusiveness: {metric_point['conclusiveness']:.2f}/10")
    print(f"  Speaker Identity: {metric_point['speaker_identity']:.2f}/10")
    print(f"  Speaker Fluidity: {metric_point['speaker_fluidity']:.2f}/10")

# Run 4-speaker conversation in 2D world
print("\\n\\nRunning 4-speaker conversation in 2D world...")
conv_2d_4speaker, metrics_2d_4speaker, positions_2d_4speaker = run_2d_world_conversation(
    iterations=30,
    participant_count=4,
    evaluation_interval=5
)

print("\\n=== 4-Speaker 2D World Results ===")
for metric_point in metrics_2d_4speaker:
    round_num = metric_point["round"]
    print(f"\\nRound {round_num}:")
    print(f"  Dynamic: {metric_point['dynamic']:.2f}/10")
    print(f"  Conclusiveness: {metric_point['conclusiveness']:.2f}/10")
    print(f"  Speaker Identity: {metric_point['speaker_identity']:.2f}/10")
    print(f"  Speaker Fluidity: {metric_point['speaker_fluidity']:.2f}/10")


## Visualizing Metric Drift

Let's plot how metrics change over the 30 iterations.


In [ ]:
def plot_metric_drift(metric_history: List[Dict], title: str):
    """Plot metric changes over time."""
    rounds = [m["round"] for m in metric_history]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(title, fontsize=14)
    
    metrics_to_plot = [
        ("dynamic", "Dynamic (Collaborative ↔ Competitive)", axes[0, 0]),
        ("conclusiveness", "Conclusiveness (Consensus ↔ Divergence)", axes[0, 1]),
        ("speaker_identity", "Speaker Identity (Similarity ↔ Diversity)", axes[1, 0]),
        ("speaker_fluidity", "Speaker Fluidity (Malleability ↔ Consistency)", axes[1, 1])
    ]
    
    for metric_key, metric_label, ax in metrics_to_plot:
        values = [m[metric_key] for m in metric_history]
        ax.plot(rounds, values, marker='o', linewidth=2, markersize=6)
        ax.set_title(metric_label)
        ax.set_xlabel("Round")
        ax.set_ylabel("Score")
        ax.set_ylim(0, 10)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot 2-speaker drift
plot_metric_drift(metrics_2d_2speaker, "2-Speaker Persona Drift in 2D World")

# Plot 4-speaker drift
plot_metric_drift(metrics_2d_4speaker, "4-Speaker Persona Drift in 2D World")


## Spatial Visualization

Let's visualize agent positions over time to see if spatial patterns emerge.


In [ ]:
def plot_agent_positions(position_history: List[Dict], title: str):
    """Plot agent positions at different time points."""
    # Plot positions at start, middle, and end
    time_points = [0, len(position_history) // 2, len(position_history) - 1]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(title, fontsize=14)
    
    for idx, time_point in enumerate(time_points):
        ax = axes[idx]
        positions = position_history[time_point]
        
        for agent_name, (x, y) in positions.items():
            ax.scatter(x, y, s=200, alpha=0.7, label=agent_name)
            ax.text(x, y, agent_name.split('_')[1], ha='center', va='center', fontsize=8)
        
        ax.set_xlim(-1, GRID_SIZE)
        ax.set_ylim(-1, GRID_SIZE)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_title(f"Round {time_point}")
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

# Plot 2-speaker positions
plot_agent_positions(positions_2d_2speaker, "2-Speaker Position Evolution")

# Plot 4-speaker positions
plot_agent_positions(positions_2d_4speaker, "4-Speaker Position Evolution")


## Comparison with Text-Only Results

### Key Observations

1. **Spatial Context Effects**: Does being in a 2D world affect persona stability?
2. **Proximity Influence**: Do agents near each other develop similar personas?
3. **Movement Patterns**: Do agents cluster or spread out?
4. **Metric Differences**: How do 2D world metrics compare to text-only (post 020)?

### 2D World vs Text-Only

**2D World (this post)**:
- Agents have spatial positions
- Proximity may influence communication
- Movement creates dynamic relationships
- Spatial context adds complexity

**Text-Only (post 020)**:
- No spatial constraints
- All agents "equally close"
- Pure conversational dynamics
- Simpler interaction model

## Summary

Persona drift in the 2D world shows:

- **Spatial patterns emerge**: Agents may cluster or maintain distance
- **Proximity effects**: Nearby agents may influence each other more
- **Metric stability**: Similar drift patterns to text-only, but with spatial variation
- **Group size matters**: 4 speakers show different dynamics than 2

The spatial context adds a new dimension to persona development:
- Agents can physically approach or avoid each other
- Spatial relationships may mirror conversational relationships
- Movement patterns may reflect persona characteristics

Future work could explore:
- How spatial proximity affects conversation topics
- Whether agents form spatial subgroups
- If movement patterns correlate with persona metrics
